<a href="https://colab.research.google.com/github/ankit4488kumar/Data-Science-Projects-Materials/blob/main/Bank_Data_Exploration/bank_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import sklearn.metrics

In [ ]:
df = pd.read_excel("bank.xlsx")

In [ ]:
df.head()

In [ ]:
def date_to_float(date_data):
    return (date_data - date_data.min())  / np.timedelta64(1,'D')


In [ ]:
df["DATE DAYS"] = date_to_float(df["VALUE DATE"])
dates = pd.Series(df["DATE"])
df["YEAR"] = dates.dt.year
df["MONTH"] = dates.dt.month
df["WEEK"] = dates.dt.isocalendar().week
df["DAY"] = dates.dt.day
df["DAYOFWEEK"] = dates.dt.dayofweek
df["Account No"] = df["Account No"].replace(df["Account No"].unique(),["A","B","C","D","E","F","G","H","I","J"])

In [ ]:
df["DATE DAYS"] = date_to_float(df["VALUE DATE"])
#dates = pd.Series(df["DATE"])
df['DATE'] = pd.to_datetime(df['DATE'])
df["YEAR"] = df['DATE'].dt.year
df["MONTH"] = df['DATE'].dt.month
df["WEEK"] = df['DATE'].dt.isocalendar().week
df["DAY"] = df['DATE'].dt.day
df["DAYOFWEEK"] = df['DATE'].dt.dayofweek


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['Account No'].nunique()

In [ ]:
df["Account No"] = df["Account No"].replace(df["Account No"].unique(),[f'A_{i}' for i in range(1,df['Account No'].nunique()+1)])

In [ ]:
df.head()

Statstical Metrices

In [ ]:
# Creating some statsticals measures

unique_accounts = df["Account No"].unique()
balance_mins = []
balance_maxs = []
balance_means = []
balance_meds = []
deposit_mins = []
deposit_maxs = []
deposit_means = []
deposit_number = []
deposit_meds = []
withdrawal_mins = []
withdrawal_maxs = []
withdrawal_means = []
withdrawal_number = []
withdrawal_meds = []
for acc in unique_accounts:
    balance_mins.append(df[df["Account No"] == acc][["BALANCE AMT"]].min().values[0])
    balance_maxs.append(df[df["Account No"] == acc][["BALANCE AMT"]].max().values[0])
    balance_means.append(df[df["Account No"] == acc][["BALANCE AMT"]].mean().values[0])
    balance_meds.append(df[df["Account No"] == acc][["BALANCE AMT"]].median().values[0])

    deposit_mins.append(df[df["Account No"] == acc][["DEPOSIT AMT"]].min().values[0])
    deposit_maxs.append(df[df["Account No"] == acc][["DEPOSIT AMT"]].max().values[0])
    deposit_means.append(df[df["Account No"] == acc][["DEPOSIT AMT"]].mean().values[0])
    deposit_number.append(df[df["Account No"] == acc][["DEPOSIT AMT"]].count().values[0])
    deposit_meds.append(df[df["Account No"] == acc][["DEPOSIT AMT"]].median().values[0])

    withdrawal_mins.append(df[df["Account No"] == acc][["WITHDRAWAL AMT"]].min().values[0])
    withdrawal_maxs.append(df[df["Account No"] == acc][["WITHDRAWAL AMT"]].max().values[0])
    withdrawal_means.append(df[df["Account No"] == acc][["WITHDRAWAL AMT"]].mean().values[0])
    withdrawal_number.append(df[df["Account No"] == acc][["WITHDRAWAL AMT"]].count().values[0])
    withdrawal_meds.append(df[df["Account No"] == acc][["WITHDRAWAL AMT"]].median().values[0])


In [ ]:
balance_mins

In [ ]:
#creating accounts data
account_data = pd.DataFrame(list(zip(unique_accounts, deposit_number, deposit_mins, deposit_maxs, deposit_means, deposit_meds, withdrawal_number, withdrawal_mins, withdrawal_maxs, withdrawal_means, withdrawal_meds, balance_mins, balance_maxs, balance_means, balance_meds)),
                           columns=["Account No","DEPOSIT N°","DEPOSIT MIN", "DEPOSIT MAX", "DEPOSIT MEAN", "DEPOSIT MEDIAN", "WITHDRAWAL N°", "WITHDRAWAL MIN", "WITHDRAWAL MAX","WITHDRAWAL MEAN", "WITHDRAWAL MEDIAN", "BALANCE MIN", "BALANCE MAX", "BALANCE MEAN", "BALANCE MEDIAN"]).round()
account_data

Statstical Exploration

In [ ]:
ax = 0
for acc in unique_accounts:
    if ax == 0:
        ax = df[df["Account No"] == acc][["Account No","VALUE DATE","BALANCE AMT"]].rename(columns={"BALANCE AMT":acc}).groupby(["VALUE DATE"]).mean().plot(ylabel="BALANCE",ylim=(-2500000000,1000000000), figsize=(10,10))
    else :
        df[df["Account No"] == acc][["Account No","VALUE DATE","BALANCE AMT"]].rename(columns={"BALANCE AMT":acc}).groupby(["VALUE DATE"]).mean().plot(ax=ax, ylabel="BALANCE", ylim=(-2500000000,1000000000), figsize=(10,10)).set_title("Accounts balance evolution over the period")


In [ ]:
#Transation Density
for acc in unique_accounts:
    df[df["Account No"] == acc][["Account No","VALUE DATE","BALANCE AMT"]].rename(columns={"BALANCE AMT": "transaction day"}).groupby(["VALUE DATE"]).mean().plot(ylabel="BALANCE",style=".-",figsize=(10,10), title="Averaged balance and transaction density over the period - Account " + acc )

Transaction periodicity:-
Ratio of Deposite percentage and Withdrawals percentage

In [ ]:
#For Deposite Percentage
deposits = []
for acc in unique_accounts:
    deposits.append(df[df["Account No"] == acc][["VALUE DATE","YEAR","MONTH","DAY","DAYOFWEEK","DEPOSIT AMT"]].rename(columns={"DEPOSIT AMT":acc+" : DEPOSIT"}).groupby(["VALUE DATE","YEAR","MONTH","DAY","DAYOFWEEK"]).count())
deposits = pd.concat(deposits)

deposits_percent = deposits
tot = deposits.groupby("MONTH").sum().sum(axis=0).to_numpy()
i = 0
for column in deposits:
    deposits_percent[column] = deposits[column]/tot[i]
    i = i + 1



In [ ]:
deposits_percent

In [ ]:
##For Withdrawals Percentage
withdrawals = []
for acc in unique_accounts:
    withdrawals.append(df[df["Account No"] == acc][["VALUE DATE","YEAR","MONTH","DAY","DAYOFWEEK","WITHDRAWAL AMT"]].rename(columns={"WITHDRAWAL AMT":acc+" : WITHDRAWAL"}).groupby(["VALUE DATE","YEAR","MONTH","DAY","DAYOFWEEK"]).count())
withdrawals = pd.concat(withdrawals)

withdrawals_percent = withdrawals
tot = withdrawals.groupby("MONTH").sum().sum(axis=0).to_numpy()
i = 0
for column in withdrawals:
    withdrawals_percent[column] = withdrawals[column]/tot[i]
    i = i + 1

In [ ]:
withdrawals_percent

In [ ]:
ax = plt.axes()
sns.heatmap(deposits_percent.groupby("YEAR").sum(), ax=ax)
ax.set_title("Accounts proportion of deposit per year")
plt.show()

ax = plt.axes()
sns.heatmap(withdrawals_percent.groupby("YEAR").sum(), ax=ax)
ax.set_title("Accounts proportion of withdrawal per year")
plt.show()

In [ ]:
def absolute_maximum_scale(series):
    return series / series.abs().max()

balances = []
for acc in unique_accounts:
    balances.append(df[df["Account No"] == acc][["VALUE DATE","YEAR","MONTH","DAY","DAYOFWEEK","BALANCE AMT"]].rename(columns={"BALANCE AMT":acc+" : BALANCE MEAN"}).groupby(["VALUE DATE","YEAR","MONTH","DAY","DAYOFWEEK"]).mean())
balances = pd.concat(balances)
balances_proportion = balances
for col in balances_proportion.columns:
    balances_proportion[col] = absolute_maximum_scale(balances[col])

In [ ]:
ax = plt.axes()
sns.heatmap(balances_proportion.groupby("MONTH").mean(), ax=ax)
ax.set_title("Balance level depending on each month of the year (normalized)")
plt.show()

In [ ]:
def balance_prediction(account_no):
    X = df[df["Account No"] == account_no][["YEAR","MONTH","WEEK","DAYOFWEEK","DAY"]].drop_duplicates().to_numpy()
    y = df[df["Account No"] == account_no][["YEAR","MONTH","WEEK","DAYOFWEEK","DAY","BALANCE AMT"]].groupby(["YEAR","MONTH","WEEK","DAYOFWEEK","DAY"]).mean().to_numpy()
    #x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25, stratify=[element > 1 for element in y.values])

    params = {
    "n_estimators": 500,
    "max_depth": 10,
    "learning_rate": 0.01,
    }
    lm = XGBRegressor()

    kf = KFold(n_splits=10, random_state=None, shuffle=True)

    for train_index , test_index in kf.split(X):
        X_train , X_test = X[train_index,:],X[test_index,:]
        y_train , y_test = y[train_index] , y[test_index]

        lm.fit(X_train,y_train)
        pred_values = lm.predict(X_test)

        r2 = f"R2 score: {r2_score(y_test[:,0], pred_values)}"
        print(r2)
        Val = pd.DataFrame({"ACTUAL": y_test[:,0], "PREDICTED": pred_values})
        Val.sort_values("ACTUAL").reset_index()[["ACTUAL", "PREDICTED"]].plot(title="Account : " + account_no, xlabel =r2)


In [ ]:
for acc in unique_accounts:
    print("\nAccount : " + acc)
    balance_prediction(acc)